In [1]:
import geopandas as gpd
import pandas as pd

# import os
# import json
# from IPython.display import display
import arcgis
from arcgis.gis import GIS, ContentManager
from arcgis.mapping import WebMap, WebScene

### Connect to ArcGIS Online (AGOL)

In [2]:
app_id = 'uPZiN9HJWneCycx3'

In [3]:
gis = GIS("https://ky-cancer.maps.arcgis.com", client_id = app_id) # log-in to the AGOL 

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ky-cancer.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=uPZiN9HJWneCycx3&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=8Fxl5Lq3tKEEid5sjVtllBuT1ngmRZ&allow_verification=false
Enter code obtained on signing in using SAML: ········


C:\Python39\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ky-cancer.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
contentManager = ContentManager(gis)

In [5]:
contentManager.search('')

[<Item title:"Hybrid County Basemap" type:Vector Tile Layer owner:todd.burus>,
 <Item title:"Behavioral Risk Factors" type:Service Definition owner:lee.park>,
 <Item title:"Basemap" type:Feature Layer Collection owner:todd.burus>,
 <Item title:"urban_region" type:Feature Layer Collection owner:todd.burus>,
 <Item title:"Environmental Indicators County" type:Feature Layer Collection owner:lee.park>,
 <Item title:"Tract Basemap" type:Feature Layer Collection owner:todd.burus>,
 <Item title:"all_site_inc_map_cover_outlined" type:Image owner:todd.burus>,
 <Item title:"ht_county : Vacancy Rate" type:Web Map owner:lee.park>,
 <Item title:"environment_county : PWS_Violations_Since_2016" type:Web Map owner:lee.park>,
 <Item title:"sociodemographics_county : Over 64" type:Web Map owner:lee.park>]

In [6]:
county = 'https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip'
gdf = gpd.read_file(county)

In [7]:
simple_gdf = gdf.loc[:, ['GEOID','geometry']]

In [8]:
simple_gdf.head(3)

,GEOID,geometry
0,31039,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53069,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35011,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."


In [9]:
simple_gdf.dtypes

GEOID         object
geometry    geometry
dtype: object

### Import our own dataset

In [10]:
import pickle
d = pickle.load(open('../dataset.pickle','rb'))
df = d['cancer_incidence']
df2 = d['economy_county']

In [28]:
df.reset_index(drop = False, inplace = True)

In [29]:
df.head(3)

Site,FIPS,County,State,Type,All Site,Bladder,Brain & ONS,Cervix,Colon & Rectum,"Corpus Uteri & Uterus, NOS",...,Liver & IBD,Lung & Bronchus,Melanoma of the Skin,Non-Hodgkin Lymphoma,Oral Cavity & Pharynx,Ovary,Pancreas,Prostate,Stomach,Thyroid
0,33001,Belknap County,New Hampshire,Incidence,519.3,26.6,6.4,NaN,41.9,42.6,...,5.7,69.1,40.3,21.5,14.6,13.1,15.3,121.4,5.5,10.4
1,33003,Carroll County,New Hampshire,Incidence,465.0,26.2,8.3,NaN,35.7,28.6,...,5.4,57.6,33.8,17.3,13.5,9.0,12.4,127.1,4.5,9.9
2,33005,Cheshire County,New Hampshire,Incidence,471.2,23.2,6.3,NaN,37.7,30.4,...,8.0,59.2,40.4,20.8,10.0,10.7,13.2,95.6,5.5,9.5


In [33]:
## merge (add geometry variable)

temp = df.merge(simple_gdf, how = 'left', left_on = 'FIPS', right_on = 'GEOID').drop('GEOID', axis =1)

In [35]:
sdf = pd.DataFrame.spatial.from_geodataframe(temp, column_name = 'geometry')

Error acquiring spatial reference from GeoDataFrame Spatial reference will not be set.'DataFrame' object has no attribute 'crs'


In [37]:
lyrs = contentManager.import_data(sdf, title = 'First Sample', folder = "Sample")

In [38]:
lyrs

<Item title:"First Sample" type:Feature Layer Collection owner:lee.park>